# **Flujo de trabajo para el dataset de radiografías de tórax**


---



### **1. Importar las librerías necesarias**

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import cv2
import shutil
import re
import cv2
import joblib
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage import io
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models

### **2. Analizar la estructura del dataset y crear un dataframe**

In [ ]:
carpeta_principal_imagenes= '/content/drive/MyDrive/Proyecto modulo 7/data'

for subcarpeta in os.listdir(carpeta_principal_imagenes):
    sub_carpeta_imagenes = os.path.join(carpeta_principal_imagenes, subcarpeta)
    if os.path.isdir(sub_carpeta_imagenes):
        print(f"Subcarpeta: {subcarpeta}")

        for sub_subcarpeta in os.listdir(sub_carpeta_imagenes):
            sub_sub_carpeta_imagenes = os.path.join(sub_carpeta_imagenes, sub_subcarpeta)
            if os.path.isdir(sub_sub_carpeta_imagenes):
                print(f"  Sub_subcarpeta: {sub_subcarpeta}")

                archivos = [f for f in os.listdir(sub_sub_carpeta_imagenes) if os.path.isfile(os.path.join(sub_sub_carpeta_imagenes, f))]
                primeros_5_archivos = archivos[:5]

                for archivo in primeros_5_archivos:
                    print(f"    Archivo: {archivo}")

In [ ]:
datos_imagenes = []

for carpeta in ['train', 'val', 'test']:
    sub_carpeta_imagenes = os.path.join(carpeta_principal_imagenes, carpeta)
    if os.path.isdir(sub_carpeta_imagenes):
        for categoria in ['neumonia', 'normal']:
            sub_carpeta_categoria = os.path.join(sub_carpeta_imagenes, categoria)
            if os.path.isdir(sub_carpeta_categoria):
                for imagen in os.listdir(sub_carpeta_categoria):
                    if imagen.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                        ruta_imagen = os.path.join(sub_carpeta_categoria, imagen)

                        with Image.open(ruta_imagen) as img:
                            ancho, alto = img.size
                            modo = img.mode
                            pixeles = list(img.getdata())

                        if 'virus' in imagen.lower():
                            sub_categoria = 'virus'
                        elif 'bacteria' in imagen.lower():
                            sub_categoria = 'bacteria'
                        else:
                            sub_categoria = 'no hay bacteria ni virus'

                        datos_imagenes.append([carpeta, categoria, sub_categoria, imagen, ancho, alto, modo,pixeles])

columnas = ['Carpeta', 'Categoría', 'Sub categoría', 'Nombre imagen', 'Ancho', 'Alto', 'Modo','Pixeles']
df = pd.DataFrame(datos_imagenes, columns=columnas)

df.to_csv('image_data.csv', index=False)

print(df.head())

  Carpeta Categoría Sub categoría                 Nombre imagen  Ancho  Alto  \
0   train  neumonia         virus     person537_virus_1067.jpeg   1112   744   
1   train  neumonia      bacteria    person53_bacteria_252.jpeg   1292   936   
2   train  neumonia      bacteria  person537_bacteria_2263.jpeg   1216  1000   
3   train  neumonia      bacteria  person540_bacteria_2272.jpeg   1088   736   
4   train  neumonia      bacteria  person543_bacteria_2281.jpeg   1088   584   

  Modo                                            Pixeles  
0    L  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1    L  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2    L  [197, 203, 206, 203, 202, 206, 209, 210, 207, ...  
3    L  [87, 82, 79, 80, 75, 65, 60, 62, 61, 65, 68, 6...  
4    L  [62, 66, 66, 66, 72, 84, 90, 88, 72, 83, 99, 9...  


In [ ]:
ruta_origen = '/content/image_data.csv'
ruta_destino = '/content/drive/MyDrive/Proyecto modulo 7/image_data.csv'

shutil.move(ruta_origen, ruta_destino)
print(f"Archivo movido a: {ruta_destino}")

### **3. División del conjunto de datos**

In [ ]:
columnas_incluidas = ['Carpeta', 'Categoría', 'Sub categoría', 'Nombre imagen', 'Ancho', 'Alto', 'Modo']

df_train = df[df['Carpeta'] == 'train'][columnas_incluidas]
df_test = df[df['Carpeta'] == 'test'][columnas_incluidas]
df_val = df[df['Carpeta'] == 'val'][columnas_incluidas]

### **4. Submuestreo de los datos de entrenamiento y test**

In [ ]:
total_imagenes_neumonia=1500
numero_imagenes_bacteria=int(total_imagenes_neumonia*0.55)
numero_imagenes_virus=int(total_imagenes_neumonia*0.45)

seleccion_imagenes_bacteria=df_train[df_train['Sub categoría']=='bacteria'].sample(n=numero_imagenes_bacteria,random_state=42)
seleccion_imagenes_virus=df_train[df_train['Sub categoría']=='virus'].sample(n=numero_imagenes_virus,random_state=42)

train_neumonia=pd.concat([seleccion_imagenes_bacteria,seleccion_imagenes_virus])

train_normal=df_train[df_train['Categoría']=='normal']

df_train_submuestreo=pd.concat([train_neumonia,train_normal])
ruta_guardado = '/content/drive/MyDrive/Proyecto modulo 7/prueba/procesamiento_de_data/df_train_acotado.joblib'
joblib.dump(df_train_submuestreo, ruta_guardado)

df_train_submuestreo.head()

In [ ]:
total_imagenes_neumonia_test=250
numero_imagenes_bacteria_test=int(total_imagenes_neumonia_test*0.55)
numero_imagenes_virus_test=int(total_imagenes_neumonia_test*0.45)

seleccion_imagenes_bacteria_test=df_test[df_test['Sub categoría']=='bacteria'].sample(n=numero_imagenes_bacteria_test,random_state=42)
seleccion_imagenes_virus_test=df_test[df_test['Sub categoría']=='virus'].sample(n=numero_imagenes_virus_test,random_state=42)

test_neumonia=pd.concat([seleccion_imagenes_bacteria_test,seleccion_imagenes_virus_test])

test_normal=df_test[df_test['Categoría']=='normal']

df_test_submuestreo=pd.concat([test_neumonia,test_normal])
ruta_guardado = '/content/drive/MyDrive/Proyecto modulo 7/prueba/df_test_acotado.joblib'
joblib.dump(df_test_submuestreo, ruta_guardado)

df_test_submuestreo.head()

### **5. Redimensionado de las imágenes y conversión a escala de grises**
Función y carpetas definidas en notebook 3. Ingeniería de características

In [ ]:
df_train_submuestreo_con_ruta['Nueva Ruta Procesada']= df_train_submuestreo_con_ruta.apply(procesar_imagen, axis=1)
df_test_submuestreo_con_ruta['Nueva Ruta Procesada']= df_test_submuestreo_con_ruta.apply(procesar_imagen, axis=1)
df_val['Ruta de Imagen Procesada'] = df_val.apply(procesar_imagen, axis=1)

### **6. Padding a las imágenes**
Función y carpetas definidas en notebook 3. Ingeniería de características

In [ ]:
image_paths_train = df_train_submuestreo_con_ruta['Nueva Ruta Procesada'].tolist()
image_paths_test = df_test_submuestreo_con_ruta['Nueva Ruta Procesada'].tolist()
image_paths_val = df_val['Ruta de Imagen Procesada'].tolist()

### **7. Redimensionamiento post padding**
Función y carpetas definidas en notebook 3. Ingeniería de características

In [ ]:
output_directory_train = '/content/drive/MyDrive/Proyecto modulo 7/prueba/data_procesada/data_para_modelo/padded/train'
image_paths_train = df_train_submuestreo_con_ruta['Nueva Ruta Procesada'].tolist()

nueva_ruta_train=procesar_imagen_padding(image_paths_train, output_directory_train)

output_directory_test = '/content/drive/MyDrive/Proyecto modulo 7/prueba/data_procesada/data_para_modelo/padded/test'
image_paths_test = df_test_submuestreo_con_ruta['Nueva Ruta Procesada'].tolist()

nueva_ruta_test=procesar_imagen_padding(image_paths_test, output_directory_test)

output_directory_val = '/content/drive/MyDrive/Proyecto modulo 7/prueba/data_procesada/data_para_modelo/padded/val'
image_paths_val = df_val['Ruta de Imagen Procesada'].tolist()

nueva_ruta_val=procesar_imagen_padding(image_paths_val, output_directory_val)

### **8. Separación de características**


In [ ]:
X_train = df_train_submuestreo_con_ruta[['Ruta Redimensionada', 'Nombre imagen']]
X_test = df_test_submuestreo_con_ruta[['Ruta Redimensionada', 'Nombre imagen']]
X_val = df_val[['Ruta Redimensionada', 'Nombre imagen']]

categoria_map = {'normal': 0, 'neumonia': 1}

y_train = df_train_submuestreo_con_ruta['Categoría'].map(categoria_map)
y_test = df_test_submuestreo_con_ruta['Categoría'].map(categoria_map)
y_val = df_val['Categoría'].map(categoria_map)

## **9. Normalizar las imágenes**
Función definidas en notebook 4. Modelo mejor versión

In [ ]:
X_train_imgs = np.array([normalizar_imagenes(os.path.join(ruta_train, img)) for img in X_train['Ruta Redimensionada']])
X_test_imgs = np.array([normalizar_imagenes(os.path.join(ruta_test, img)) for img in X_test['Ruta Redimensionada']])
X_val_imgs = np.array([normalizar_imagenes(os.path.join(ruta_val, img)) for img in X_val['Ruta Redimensionada']])

### **10. Definir y entrenar el modelo**
Modelo definido DenseNet en notebook 4. Modelo mejor versión

In [ ]:
class CustomDenseNet(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomDenseNet, self).__init__()
        self.densenet = models.densenet121(pretrained=True)
        self.densenet.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_ftrs = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.densenet(x)

model = CustomDenseNet(num_classes=2)

num_epochs = 25
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def calculate_accuracy(y_pred, y_true):
    _, predicted = torch.max(y_pred, 1)
    correct = (predicted == y_true).sum().item()
    total = y_true.size(0)
    accuracy = correct / total
    return accuracy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

best_val_accuracy = 0.0
patience = 5
no_improvement_count = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_accuracy = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        inputs = inputs.unsqueeze(1)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        total_accuracy += calculate_accuracy(outputs, labels)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = total_accuracy / len(train_loader)

    print(f"Época {epoch+1}/{num_epochs}, Pérdida en conjunto de entrenamiento: {epoch_loss:.4f}, Precisión en conjunto de entrenamiento: {epoch_accuracy:.4f}")

### **11. Evaluar el modelo**

In [ ]:
    model.eval()
    with torch.no_grad():
        test_running_loss = 0.0
        test_total_accuracy = 0.0

        for test_inputs, test_labels in test_loader:
            test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
            test_inputs = test_inputs.unsqueeze(1)

            test_outputs = model(test_inputs)
            test_loss = criterion(test_outputs, test_labels)
            test_running_loss += test_loss.item()
            test_total_accuracy += calculate_accuracy(test_outputs, test_labels)

        test_epoch_loss = test_running_loss / len(test_loader)
        test_epoch_accuracy = test_total_accuracy / len(test_loader)

        print(f"Época {epoch+1}/{num_epochs}, Pérdida en conjunto de prueba: {test_epoch_loss:.4f}, Precisión en conjunto de prueba: {test_epoch_accuracy:.4f}")


**Resumen del flujo de trabajo**

1. Importar las librerías necesarias
2. Analizar la estructura del dataset y crear un dataframe
3. División del conjunto de datos
4. Submuestreo de los datos de entrenamiento y test
5. Redimensionado de las imágenes y conversión a escala de grises
6. Padding a las imágenes
7. Redimensionamiento post padding
8. Separación de características
9. Normalizar las imágenes
10. Definir y entrenar el modelo
11. Evaluar el modelo